In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np

from segment_speed_utils import helpers, gtfs_schedule_wrangling
from shared_utils import rt_utils, portfolio_utils

import sys
sys.path.append('./scripts/')

from calitp_data_analysis import geography_utils

In [3]:
from update_vars import (analysis_date, EXPORT_PATH, GCS_FILE_PATH, PROJECT_CRS,
SEGMENT_BUFFER_METERS, HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD, SHARED_STOP_THRESHOLD,
                        HALF_MILE_BUFFER_METERS, INTERSECTION_BUFFER_METERS, HQTA_SEGMENT_LENGTH)

In [5]:
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
gcsgp = GCSGeoPandas()

In [51]:
import intake
catalog = intake.open_catalog("*.yml")

# Debug

## Major Stop Overcounts near UCLA

* sepulveda: present in spatial only
* san vicente: not present
* wilshire: present in spatial only (both condo canyon and BH)
* pico: not present

In [52]:
ms_spatial = catalog.major_stop_bus.read()

In [53]:
ms_branching = catalog.major_stop_bus_branching.read()

In [56]:
# ms_branching.explore()

In [73]:
# ms_spatial.explore()

In [59]:
path = f"{GCS_FILE_PATH}pairwise.parquet"

In [64]:
df = pd.read_parquet(path)

In [65]:
df.head(3)

,hqta_segment_id,intersect_hqta_segment_id,fwd_azimuth_360,intersect_fwd_azimuth_360,intersect
0,335745,1122111018,47.530306,25.693099,0.0
1,4151130162,1122111018,57.151748,25.693099,0.0
2,335745,2985449236,47.530306,16.429054,0.0


In [80]:
df.shape

(34868, 5)

In [85]:
intersects = df.query('intersect == 1')

In [86]:
intersects

,hqta_segment_id,intersect_hqta_segment_id,fwd_azimuth_360,intersect_fwd_azimuth_360,intersect
4,7891039,336677357,50.209742,322.887452,1.0
5,2888474494,336677357,50.513496,322.887452,1.0
6,7891039,406991407,50.209742,320.164468,1.0
7,2888474494,406991407,50.513496,320.164468,1.0
8,7891039,2322979918,50.209742,319.509786,1.0
...,...,...,...,...,...
34863,3266363212,4007624370,313.855499,234.264878,1.0
34864,3266363212,2274171199,313.855499,234.264878,1.0
34865,3266363212,3149890812,313.855499,225.118313,1.0
34866,3266363212,2375090011,313.855499,225.140364,1.0


In [66]:
path = f"{GCS_FILE_PATH}subset_corridors.parquet"

In [68]:
corridors = gcsgp.read_parquet(path)

In [75]:
metro_corr = corridors.query('schedule_gtfs_dataset_key.str.contains("42610e")')

In [76]:
m = metro_corr.explore(column='fwd_azimuth')

In [77]:
metro_ms = ms_spatial.query('schedule_gtfs_dataset_key_primary.str.contains("42610e")')

In [94]:
# metro_ms.explore(color='red', m=m)

In [91]:
inter_corr = corridors.query('hqta_segment_id.isin(@intersects.hqta_segment_id)')

In [92]:
m2 = inter_corr.explore(column='fwd_azimuth')

In [95]:
# metro_ms.explore(color='red', m=m2)

## bug theory

* overlapping routes within a single operator are getting tagged as intersections, apparently regardless of azimuth